<a href="https://colab.research.google.com/github/henriquepgomide/caRtola/blob/master/caRtola_como_ler_reposit%C3%B3rio_do_github_com_BeautifulSoup_e_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src="https://henriquepgomide.github.io/cartola-supertrunfo/img/pfc-logo.7412844e.png" alt="drawing" width="100"
</div>

# Como ler e agregar os dados do repositório caRtola em Python com BeautifulSoup e Pandas.


Por Henrique Gomide, Ph.D.




## Introdução e objetivos

Estamos quase prontos para começarmos o Brasileirao 2020 e o Cartola FC. Habemus Cartola! Neste primeiro post do ano você aprenderá a ler os dados de todas as rodadas de um determinado ano. Para isso, iremos:


*   Usar o beautifulsoup para raspar páginas HTML
*   Combinar múltiplos DataFrames do pandas
*   Criar uma função genérica para baixar os dados de 2018-2020!

Vamos lá!


In [1]:
# Importar bibliotecas
import re                          # Expressão regulares
import requests                    # Acessar páginas da internet
from bs4 import BeautifulSoup      # Raspar elementos de páginas da internet
import pandas as pd                # Abrir e concatenar bancos de dados

## Encontrar arquivos do repositório

Com o repositório [caRtola]() já temos onde encontrar os dados entre os anos 2014 e 2020. Uma primeira solução seria clonar o repositório com todos os dados e abrir no Excel.

No entanto, podemos usar o Python em nosso favor. Para começar vamos nos concentrar no ano de 2019. O link da página é: https://github.com/henriquepgomide/caRtola/tree/master/data/2019

<br>

![Imagem do repositório](https://i.imgur.com/wJDbju4.png)

<br>

Nos temos nesta página da web, os nomes dos arquivos - seguem o padrão rodada* - que precisamos abrir. 

## Usar BeautifulSoup para raspar nomes dos arquivos

Agora, precisamos baixar esta página para pegarmos esta lista de nomes. Para isso, usaremos as bibliotecas ```requests``` e ```BeautifulSoup```. Para nossa sorte, elas já estão instaladas no Colab.

In [2]:
# URL com caminho do repositório
URL = 'https://github.com/henriquepgomide/caRtola/tree/master/data/2019'
html = requests.get(URL)

In [3]:
# Criar objeto BeautifulSoup para raspar urls 
soup = BeautifulSoup(html.text, 'lxml')

In [ ]:
# Imprimir soup
soup

Conseguimos capturar todo o html. No entanto, ainda precisamos selecionar as partes do html que possuem links.

As tags de links geralmente tem esta estrutura:

```<a href="http://alguma.coisa.com/arquivo-texto">Texto</a>```

Com base em nosso conhecimento primário de html, já sabemos que podemos filtrar as tags de link ```a``` que possuem determinados padrões dentro de href.

Isso é fácil de fazer usando a função ```find_all``` do BeatutifulSoup.

```
find_all('a', href=True)
```

In [5]:
result = []
for tag in soup.find_all('a', href=True):
    result.append(tag)
result[70:90:2]

[<a class="link-gray ml-2" data-pjax="" href="/henriquepgomide/caRtola/commit/b4283c61ddc8fcf4f70ded2f8a911850a8b47753">
 <relative-time class="no-wrap" datetime="2019-12-09T22:39:14Z">Dec 9, 2019</relative-time>
 </a>,
 <a class="pl-3 pr-3 py-3 p-md-0 mt-n3 mb-n3 mr-n3 m-md-0 link-gray-dark no-underline no-wrap" data-pjax="" href="/henriquepgomide/caRtola/commits/master/data/2019">
 <svg aria-hidden="true" class="octicon octicon-history text-gray" height="16" text="gray" version="1.1" viewbox="0 0 16 16" width="16"><path d="M1.643 3.143L.427 1.927A.25.25 0 000 2.104V5.75c0 .138.112.25.25.25h3.646a.25.25 0 00.177-.427L2.715 4.215a6.5 6.5 0 11-1.18 4.458.75.75 0 10-1.493.154 8.001 8.001 0 101.6-5.684zM7.75 4a.75.75 0 01.75.75v2.992l2.028.812a.75.75 0 01-.557 1.392l-2.5-1A.75.75 0 017 8.25v-3.5A.75.75 0 017.75 4z" fill-rule="evenodd"></path></svg>
 <span class="d-none d-sm-inline">
 <strong>History</strong>
 </span>
 </a>,
 <a class="js-navigation-open d-block py-2 px-3" href="/henriquep

Temos muitos links nesta página. Mas nosso interesse está nos últimos links da saída anterior. Repare que eles seguem um padrão. O que é campo fértil para Expressões regulares. REGEX, hora de aquecer e entrar em campo. 


Podemos usar diversas expressões regulares para pegarmos o que quisermos. Neste tutorial, iremos nos concentrar no texto dentro do atributo *href*.

```
<a class="js-navigation-open link-gray-dark" href="/henriquepgomide/caRtola/tree/master/data/2019/team-rankings" id="4c3fa43e2bc40689660a8b10cb303c0a-4c01b0b9ac804062a18eb94535e9bae3cbdeb0ac" title="team-rankings">team-rankings</a>
```

Este link possui um lugar para o [treino e teste de REGEX](https://regex101.com/). Recomendo.

## Combinar múltiplos dataframes do Pandas

In [6]:
 '''
 Selecionar aqueles href que:
  a) possuem o padrão rodada-[número de um ou dois dígitos] 
  b) terminam com csv. 
'''
regex =  'rodada-([0-9]|[0-9][0-9])\.csv'

Além de selecionar as tags com href, vamos aproveitar e criar um dicionário com o nome dos arquivos e com o caminho dos arquivos **raw** do github. Isso nos possibilitará: 

1. Criar uma coluna com a informação da rodada no pandas
2. Iterar sobre uma lista de urls e baixar tudo numa lista de DataFrames do Pandas

In [7]:
dict_of_files = {}                                                          # Criar dicionário vazio
for tag in soup.find_all('a', attrs={'href': re.compile(regex)}):           # Encontrar tags de nosso interesse
    href_str = tag.get('href')                                              
    file_name = re.sub('/henriquepgomide/caRtola/blob/master/data/2019/',   # Substituir padrão por nada
                       '', 
                       href_str)
    
    file_url = re.sub('/henriquepgomide/caRtola/blob/master/data/2019/',    # Substituir padrão por links para arquivos raw
                    'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2019/', 
                    href_str)

    dict_of_files[file_name] = file_url

In [8]:
# Imprimir os primeiros casos do dicionário criado
dict(list(dict_of_files.items())[0:4]) # Ignore este código horrível

{'rodada-1.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2019/rodada-1.csv',
 'rodada-10.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2019/rodada-10.csv',
 'rodada-11.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2019/rodada-11.csv',
 'rodada-12.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2019/rodada-12.csv'}

In [9]:
# Ler os dataframes dos arquivos
list_of_dataframes = []
for key, item in dict_of_files.items():
    df = pd.read_csv(item)
    df['rodada'] = key
    list_of_dataframes.append(df)

In [13]:
# Combinar lista de DataFrames criados
cartola = pd.concat(list_of_dataframes)
cartola.shape

(30581, 34)

## Testar importação

Hora de testar nossa implementação e verificar se os dados estão okay. Para isso, vamos usar as funções clássicas do pandas.

In [12]:
# Ver informações do dataframe
cartola.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30581 entries, 0 to 774
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  30581 non-null  int64  
 1   atletas.nome                30581 non-null  object 
 2   atletas.slug                30581 non-null  object 
 3   atletas.apelido             30581 non-null  object 
 4   atletas.foto                29741 non-null  object 
 5   atletas.atleta_id           30581 non-null  int64  
 6   atletas.rodada_id           30581 non-null  int64  
 7   atletas.clube_id            30581 non-null  int64  
 8   atletas.posicao_id          30581 non-null  object 
 9   atletas.status_id           30581 non-null  object 
 10  atletas.pontos_num          30581 non-null  float64
 11  atletas.preco_num           30581 non-null  float64
 12  atletas.variacao_num        30581 non-null  float64
 13  atletas.media_num           30581

In [11]:
# Ver estrutura de dados
cartola.head()

,Unnamed: 0,atletas.nome,atletas.slug,atletas.apelido,atletas.foto,atletas.atleta_id,atletas.rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.clube.id.full.name,FS,RB,PE,FC,G,FF,FT,FD,DD,GS,SG,A,CA,I,CV,PP,GC,DP,rodada
0,1,Glaybson Yago Souza Lisboa,yago-pikachu,Yago Pikachu,https://s.glbimg.com/es/sde/f/2018/09/27/ade8a...,80196,1,267,mei,Provável,2.0,11.1,-2.9,2.0,Vasco,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-1.csv
1,2,Juan Ramón Cazares Sevillano,cazares,Cazares,https://s.glbimg.com/es/sde/f/2018/05/07/2952d...,81682,1,282,mei,Contundido,0.0,11.0,0.0,0.0,Atlético-MG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-1.csv
2,3,Alessandro Vinícius Gonçalves da Silva,alessandro-vinicius,Alessandro Vinícius,https://s.glbimg.com/es/sde/f/2019/03/30/ffa3d...,94495,1,282,mei,Nulo,0.0,1.0,0.0,0.0,Atlético-MG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-1.csv
3,4,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2019/04/26/7bdef...,38509,1,262,gol,Contundido,0.0,5.0,0.0,0.0,Flamengo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-1.csv
4,5,Réver Humberto Alves Araújo,rever,Réver,https://s.glbimg.com/es/sde/f/2019/03/29/d49f2...,52253,1,282,zag,Provável,0.0,16.0,0.0,0.0,Atlético-MG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-1.csv


## Desafio - função

Já temos o código pronto para 2019. Que tal escrever uma função para coletar os dados dos anos 2018, 2019 e 2020?

Agora é com você...

In [27]:
# Sua resposta....

**Solução**: Clique **aqui** para ver uma resposta.

<!--
# Minha resposta
# A sua provavelmente será melhor :)

def read_cartola_data(year):
    '''
    Read data from a given year of the CaRtola repository

    Parameters:
    year (int) - year inside the range 2018-2020.
    ''' 

    if year in [2018, 2019, 2020]:

        # URL para baixar os arquivos
        url = 'https://github.com/henriquepgomide/caRtola/tree/master/data/{}'.format(year)
        html = requests.get(url)
    
        soup = BeautifulSoup(html.text, 'lxml')
    
        dict_of_files = {}
        for tag in soup.find_all('a', attrs={'href': re.compile('rodada-([0-9]|[0-9][0-9])\.csv')}):
            href_str = tag.get('href')
            file_name = re.sub('/henriquepgomide/caRtola/blob/master/data/{}/'.format(year), 
                            '', 
                            href_str)
            
            file_url = re.sub('/henriquepgomide/caRtola/blob/master/data/{}/'.format(year), 
                            'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/{}/'.format(year), 
                            href_str)
            dict_of_files[file_name] = file_url
    
        list_of_dataframes = []
        for key, item in dict_of_files.items():
            df = pd.read_csv(item)
            df['rodada'] = key
            list_of_dataframes.append(df)
    
        df_cartola = pd.concat(list_of_dataframes)
    
        return df_cartola
    
    else:
        print('You need to add an year within the range: 2018 and 2020')

-->

## Função para baixar os dados de diversos anos

## Happy Coding!


<div>
<img src="https://www.openmoji.org/data/color/svg/1F913.svg" width="200">
</div>